<a href="https://colab.research.google.com/github/dst3v3n/Modelos-AI/blob/main/MML_LogR_HACKATHON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de regresion logistica

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Configuración para visualización
sns.set(style="whitegrid")

In [2]:
df = pd.read_csv(r'datos_limpioscsv.csv')

# Mostrar las primeras filas para verificar
df.head()

,nacimiento,genero,industria,sector,nucleo,em_fom_remoto,em_fom_camb_remoto,t_dez_remoto,t_casa,t_familia,t_remoto_3meses,t_pref_remoto_3meses,h_trabajo_remoto
0,1976,1,13,31,1,2,2,0.0,1.0,7.0,0.5,0.8,8.0
1,1971,1,5,31,2,1,1,0.0,4.0,4.0,0.1,0.5,10.0
2,1974,1,8,25,1,2,2,0.0,3.0,4.0,1.0,1.0,9.0
3,1985,1,11,37,1,1,1,2.0,4.0,2.0,0.5,0.4,8.0
4,1992,2,2,2,2,2,2,0.5,0.5,5.0,1.0,1.0,9.0


### Categorizacion

In [3]:
# Definir las categorías de productividad
def categorize_productivity(hours):
    if hours <= 4.5:
        return 'Baja'
    elif hours <= 7.5:
        return 'Moderado'
    else:
        return 'Alta'

# Aplicar la categorización a la variable objetivo
df['productividad'] = df['h_trabajo_remoto'].apply(categorize_productivity)

# Selección de variables predictoras (X) y la variable objetivo (y)
X = df[['nacimiento', 'genero', 'industria' , 'sector' , 'nucleo' , 'em_fom_camb_remoto' , 't_dez_remoto' , 't_casa' , 't_familia' , 't_remoto_3meses' ]]
y = df['productividad']

# Codificar las etiquetas categóricas
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Fit on the original labels


### Entrenamiento y evaluacion

In [4]:

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Escalado de las variables
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Entrenamiento del modelo
#model = LogisticRegression()
model = LogisticRegression(multi_class='ovr')  # 'ovr' para clasificación uno contra el resto, o 'multinomial'
model.fit(X_train, y_train_encoded)  # Use encoded labels for training

# Predicciones y evaluación
y_pred_encoded = model.predict(X_test)  # Predictions are now encoded

# Decodificar las predicciones
y_pred_decoded = label_encoder.inverse_transform(y_pred_encoded)  # Inverse transform encoded predictions
y_test_decoded = label_encoder.inverse_transform(y_test_encoded)  # Inverse transform encoded test labels


### matriz de confusion y reporte de clasificacion

In [5]:
print(confusion_matrix(y_test_decoded, y_pred_decoded))
print(classification_report(y_test_decoded, y_pred_decoded))

[[616   1  21]
 [  3  21  26]
 [136   5  79]]
              precision    recall  f1-score   support

        Alta       0.82      0.97      0.88       638
        Baja       0.78      0.42      0.55        50
    Moderado       0.63      0.36      0.46       220

    accuracy                           0.79       908
   macro avg       0.74      0.58      0.63       908
weighted avg       0.77      0.79      0.76       908



## Pasos para importar, exportar y probar en la aplicacion web

In [6]:
import joblib

# Guardar el modelo
joblib.dump(model, 'modelo_logistico.pkl')
# Guardar el escalador
joblib.dump(scaler, 'escalador.pkl')
# Guardar el codificador de etiquetas
joblib.dump(label_encoder, 'codificador_etiquetas.pkl')


['codificador_etiquetas.pkl']

In [7]:
# Cargar el modelo
modelo_cargado = joblib.load('modelo_logistico.pkl')
# Cargar el escalador
escalador_cargado = joblib.load('escalador.pkl')
# Cargar el codificador de etiquetas
codificador_etiquetas_cargado = joblib.load('codificador_etiquetas.pkl')

### como meter datos

In [8]:
.def recoger_datos():
    print("Por favor, ingresa los siguientes datos (todos son números):")

    nacimiento = int(input("Año de nacimiento: "))
    genero = int(input("Género 1 a 4: "))
    industria = int(input("Industria (1 a 19): "))
    sector = int(input("Sector (1 a 42): "))
    nucleo = int(input("Núcleo (1 a 7): "))
    em_fom_camb_remoto = int(input("Empleador fomenta cambios para apoyar el trabajo remoto (1 a 5): "))
    t_dez_remoto = int(input("Tiempo deseado al trabajo remoto (en horas max 24): "))
    t_casa = int(input("Tiempo dedicado a tareas del hogar (en horas): "))
    t_familia = int(input("Tiempo dedicado a la familia (en horas): "))

    # Handle potential comma as decimal separator
    t_remoto_3meses_input = input("Tiempo de trabajo remoto en los últimos 3 meses (en % de 0 a 100): ")
    t_remoto_3meses = int(float(t_remoto_3meses_input.replace(',', '.')))  # Replace comma with dot, then convert to float, then to int

    nuevos_datos = [[nacimiento, genero, industria, sector, nucleo, em_fom_camb_remoto, t_dez_remoto, t_casa, t_familia, t_remoto_3meses]]

    return nuevos_datos

# Recoger los datos
nuevos_datos = recoger_datos()
print("Datos recogidos:", nuevos_datos)

Por favor, ingresa los siguientes datos (todos son números):
Año de nacimiento: 2000
Género 1 a 4: 2
Industria (1 a 19): 8
Sector (1 a 42): 39
Núcleo (1 a 7): 7
Empleador fomenta cambios para apoyar el trabajo remoto (1 a 5): 4
Tiempo deseado al trabajo remoto (en horas max 24): 8
Tiempo dedicado a tareas del hogar (en horas): 1
Tiempo dedicado a la familia (en horas): 2
Tiempo de trabajo remoto en los últimos 3 meses (en % de 0 a 100): 0.7
Datos recogidos: [[2000, 2, 8, 39, 7, 4, 8, 1, 2, 0]]


### como predecir con nuevos datos y salida esperada

In [9]:
# Nuevos datos (ejemplo)

# Escalar los nuevos datos
nuevos_datos_escalados = escalador_cargado.transform(nuevos_datos)

# Hacer predicciones
predicciones_codificadas = modelo_cargado.predict(nuevos_datos_escalados)

# Decodificar las predicciones
predicciones_decodificadas = codificador_etiquetas_cargado.inverse_transform(predicciones_codificadas)

print(predicciones_decodificadas)

['Alta']


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
